In [2]:
import json
import os
import pandas as pd

jsonFOlderPath='D:\\PhD\\Dataset\\CourtOrdersJson\\DistrictandSessionCourtPune\\CriBailAppln'

df = pd.DataFrame()

for subdir, dirs, files in os.walk(jsonFOlderPath):
    for filename in files:
        filepath = subdir + os.sep + filename

        if filepath.endswith(".json"):            
            f = open(filepath,"r", encoding="utf8")
            data = json.load(f) 
            df=df.append(data, ignore_index = True)

In [3]:
print(df.shape)
print(type(df))
print(df.columns)

(470, 12)
<class 'pandas.core.frame.DataFrame'>
Index(['actualOrders', 'caseNumber', 'caseType', 'caseYear',
       'courtEstablishment', 'decisionReasons', 'decisions', 'id',
       'judgements', 'parties', 'rawTextOrder', 'title'],
      dtype='object')


# Preparing the list of judgment points with Legal references

In [5]:
all_judgements_all_docs=df['judgements']
print(type(all_judgements_all_docs))

leg_ref_judgementPoints=[]
for all_judgements_one_docs in all_judgements_all_docs:
    for judgement in all_judgements_one_docs:        
        if 1==judgement['judgementSequence']:
            #print(judgement['judgementSequence'])
            #print(judgement['judgementPoint'])
            leg_ref_judgementPoints.append(judgement['judgementPoint'])
        
print(len(leg_ref_judgementPoints))
print(leg_ref_judgementPoints[0])

<class 'pandas.core.series.Series'>
470
Applicant ­ Rahul Sakharam Ingle has filed this application under Section 438 of the Code of Criminal Procedure for the grant of Anticipatory Bail in connection with C.R.No. 217/2018 under Section 323, 326, 336, 337, 504 and 506 of the Indian Penal Code.


In [149]:
import re

sectionReferenceRegex = '(Sections |Section |sections |section |(u/s )|sec )'
num = '(\d+)?'
listOfNumbers = '((\d+,\s+)+\d+)?'
andRegex = '(\s*and\s*)?'
readWith = '( r/w\. | r/w )?'
ofTheConst = '( of the)?'
legalReferenceRegex = re.compile(sectionReferenceRegex+listOfNumbers+andRegex+num+readWith+ofTheConst)

matched=0
unmatched=0
i=0
for judgementPoint in leg_ref_judgementPoints:
    i=i+1
    print("\n\n",judgementPoint)
    legRef = legalReferenceRegex.findall(judgementPoint)    
    if None != legRef:
        print('Legal reference found: ',i , " :", legRef[1])
        print([(x.start(),x.end()) for x in re.finditer(legalReferenceRegex, judgementPoint)])
        matched=matched+1
    else:        
        #print('Legal reference not found')
        unmatched=unmatched+1
        
print("Matched docs: ",matched," Unmatched docs: ",unmatched," Total docs: ",i)



 Applicant ­ Rahul Sakharam Ingle has filed this application under Section 438 of the Code of Criminal Procedure for the grant of Anticipatory Bail in connection with C.R.No. 217/2018 under Section 323, 326, 336, 337, 504 and 506 of the Indian Penal Code.
Legal reference found:  1  : ('Section ', '', '323, 326, 336, 337, 504', '337, ', ' and ', '506', '', ' of the')
[(66, 84), (189, 235)]


 1] Applicants­Bhimraj Gangaram Nath and Janak Ratna Dhakal have filed this application for bail under section 439 of Cr.P.C. in connection with CR No.588/2018 registered with Sangvi Police Station, Pune, under section 399, 402 of IPC, under section 37 (1) r/w. S. 135 of Bombay Police Act and under section 4 (25) of Arms Act.
Legal reference found:  2  : ('section ', '', '399, 402', '399, ', '', '', '', '')
[(102, 113), (210, 226), (241, 251), (299, 308)]


 1] Perused application and say filed by A.P.P. for State. Perused documents. Heard Advocate for applicants and A.P.P. for State.


IndexError: list index out of range